In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_DATA_DIR = '/classes/2080001_spr2022/Data/'

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ilai0/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Wordnet

In [4]:
from nltk.corpus import wordnet as wn
poses = {'n':'noun','v':'verb','s':'adj(s)','a':'adj','r':'adv'}
for synset in wn.synsets('good'):
    print("{}:{}".format(poses[synset.pos()],
                         ",".join([l.name() for l in synset.lemmas()])))

noun:good
noun:good,goodness
noun:good,goodness
noun:commodity,trade_good,good
adj:good
adj(s):full,good
adj:good
adj(s):estimable,good,honorable,respectable
adj(s):beneficial,good
adj(s):good
adj(s):good,just,upright
adj(s):adept,expert,good,practiced,proficient,skillful,skilful
adj(s):good
adj(s):dear,good,near
adj(s):dependable,good,safe,secure
adj(s):good,right,ripe
adj(s):good,well
adj(s):effective,good,in_effect,in_force
adj(s):good
adj(s):good,serious
adj(s):good,sound
adj(s):good,salutary
adj(s):good,honest
adj(s):good,undecomposed,unspoiled,unspoilt
adj(s):good
adv:well,good
adv:thoroughly,soundly,good


In [5]:
from nltk.corpus import wordnet as wn
poses = {'n':'noun','v':'verb','s':'adj(s)','a':'adj','r':'adv'}
for synset in wn.synsets('bank'):
    print("{}:{}".format(poses[synset.pos()],
                         ",".join([l.name() for l in synset.lemmas()])))

noun:bank
noun:depository_financial_institution,bank,banking_concern,banking_company
noun:bank
noun:bank
noun:bank
noun:bank
noun:bank,cant,camber
noun:savings_bank,coin_bank,money_box,bank
noun:bank,bank_building
noun:bank
verb:bank
verb:bank
verb:bank
verb:bank
verb:bank
verb:deposit,bank
verb:bank
verb:trust,swear,rely,bank


In [6]:
from nltk.corpus import wordnet as wn
panda = wn.synset("panda.n.01")
hyper = lambda s: s.hypernyms()
list(panda.closure(hyper))

[Synset('procyonid.n.01'),
 Synset('carnivore.n.01'),
 Synset('placental.n.01'),
 Synset('mammal.n.01'),
 Synset('vertebrate.n.01'),
 Synset('chordate.n.01'),
 Synset('animal.n.01'),
 Synset('organism.n.01'),
 Synset('living_thing.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

# We8there

In [95]:
import gensim 
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
# in terminal: python -m spacy download en_core_web_md
#from pyLDAvis import gensimvis
import pyLDAvis.gensim as gensimvis
import future
# lateset verion
# import pyLDAvis.gensim_models as gensimvis
import re
import string
import operator
from collections import defaultdict
import pprint
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # pip install vaderSentiment

In [89]:
pyLDAvis.__version__

'3.2.2'

In [90]:
!pip install --upgrade pyLDAvis

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyLDAvis-3.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
texts = np.load(_DATA_DIR+"we8there_texts.npy",allow_pickle=True)

In [13]:
dictionary     = gensim.corpora.Dictionary(texts)
bow_corpus     = [dictionary.doc2bow(doc) for doc in texts]

In [14]:
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,id2word=dictionary,num_topics=10, passes=20,random_state = 1)

In [15]:
# display the topics we've found
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    vis_data1 = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(vis_data1)

In [16]:
# list all topics: we choose topic 10 which means each topic concludes 10 variables.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"veri nice" + 0.012*"veri friend" + 0.010*"veri good" + 0.008*"came out" + 0.008*"veri pleasant" + 0.007*"make feel" + 0.007*"staff veri" + 0.006*"server veri" + 0.006*"atmospher veri" + 0.005*"out world"
Topic: 1 
Words: 0.018*"veri good" + 0.012*"can eat" + 0.009*"ice cream" + 0.007*"everi time" + 0.006*"bad experi" + 0.006*"sever time" + 0.005*"food veri" + 0.005*"high recommend" + 0.005*"breakfast lunch" + 0.005*"least onc"
Topic: 2 
Words: 0.007*"take out" + 0.006*"main cours" + 0.006*"san francisco" + 0.006*"mash potato" + 0.005*"well worth" + 0.005*"wide varieti" + 0.005*"fri rice" + 0.005*"can go" + 0.004*"dine room" + 0.004*"can wait"
Topic: 3 
Words: 0.018*"dine experi" + 0.017*"veri good" + 0.010*"food veri" + 0.008*"veri reason" + 0.008*"veri nice" + 0.006*"food servic" + 0.006*"alway great" + 0.006*"mexican food" + 0.005*"crab cake" + 0.005*"servic alway"
Topic: 4 
Words: 0.014*"dine room" + 0.011*"realli good" + 0.010*"ice cream" + 0.009*"pretti goo

# Trump&Tweets

In [17]:
df = pd.read_csv(_DATA_DIR+"trump.csv")

In [18]:
df.head()

,date,text
0,9/26/19 2:46,One of our best fundraising days EVER!
1,9/26/19 2:45,So true but it will never work!
2,9/25/19 20:17,I have informed @GOPLeader Kevin McCarthy and ...
3,9/25/19 18:13,Wow! “Ukraine Whistleblower’s lead attorney do...
4,9/25/19 15:56,“He (President Trump) didn’t specifically ment...


In [19]:
# print out how many tweets we collected
text_data = list(df["text"].values)
date      = list(df["date"].values)
T         = len(text_data) # 8411 tweets remaining

In [32]:
# !python3.6 -m spacy download en_core_web_md

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [33]:

spacy_nlp   = spacy.load('en_core_web_md')

In [34]:
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [35]:
def clean_tweets(tweet):
    """
    Tokenize and lemmatize an input tweet
    Args:
        tweet: string
    Returns:
        a list containing tokens
    """
    tweet       = re.sub('&amp;', ' ',tweet)
    tweet       = emoji_pattern.sub(r' ', tweet)
    word_tokens = spacy_nlp(tweet)
    tokens      = []
    for w in word_tokens:
        if not w.is_stop: # not stop words
            s   = w.text.lower()
#            print(s)
#            if s[0] == '@' or s[0] == '#':
#                continue
            s   = re.sub(r'^[@#]', '', s)
            s   = re.sub(r'[^a-zA-Z0-9_]+$', '', s)
            s   = re.sub(r'[-,#()@=!\"\'\?\/:]+', ' ', s)
            #replace consecutive non-ASCII characters with a space
            s   = re.sub(r'[^\x00-\x7F]+',' ', s)
            tokens += s.split()
    text = " ".join(tokens)
#    print(tokens)
    word_tokens    = spacy_nlp(text)
    filtered_tweet = []
    for w in word_tokens:
        if not w.is_stop:
            if w.lemma_ != "-PRON-":
                s = w.lemma_.lower()
            else:
                s = w.lower_
            s = s.strip('-')
            if re.match(r'^[a-zA-Z_\.]+$', s):
                filtered_tweet.append(s)
#            if s not in string.punctuation and not re.match(r"^[0-9\.]+$", s) and re.match(r'[a-zA-Z_]+', s):
#                filtered_tweet.append((s, w.pos_))
    
    return filtered_tweet

In [36]:
# bag-of-words model
# clean each tweet in text data and then store them as processed_docs
processed_docs = []
for t in text_data:
    try: 
        processed_docs.append(clean_tweets(t))
    except: pass


# create a dictionary for all tweets
dictionary     = gensim.corpora.Dictionary(processed_docs)
# create word bow for all tweets, this can help us record the times of each word appeared in each tweet.
bow_corpus     = [dictionary.doc2bow(doc) for doc in processed_docs]

In [38]:
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,id2word=dictionary,num_topics=10, passes=20,random_state = 1)

In [39]:
# list all topics: we choose topic 10 which means each topic concludes 10 variables.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.031*"great" + 0.011*"state" + 0.010*"endorsement" + 0.010*"total" + 0.010*"strong" + 0.009*"job" + 0.009*"people" + 0.009*"win" + 0.008*"military" + 0.008*"crime"
Topic: 1 
Words: 0.016*"fbi" + 0.012*"comey" + 0.009*"trump" + 0.008*"medium" + 0.008*"clinton" + 0.007*"say" + 0.007*"president" + 0.007*"news" + 0.007*"lie" + 0.007*"people"
Topic: 2 
Words: 0.027*"vote" + 0.018*"big" + 0.017*"great" + 0.015*"republican" + 0.013*"tax" + 0.013*"cut" + 0.012*"win" + 0.011*"bill" + 0.011*"house" + 0.010*"republicans"
Topic: 3 
Words: 0.011*"good" + 0.010*"great" + 0.009*"president" + 0.009*"state" + 0.008*"news" + 0.007*"bad" + 0.006*"new" + 0.006*"time" + 0.006*"go" + 0.006*"know"
Topic: 4 
Words: 0.042*"great" + 0.019*"thank" + 0.012*"work" + 0.010*"honor" + 0.010*"minister" + 0.010*"prime" + 0.010*"today" + 0.009*"happy" + 0.009*"look" + 0.009*"forward"
Topic: 5 
Words: 0.017*"china" + 0.016*"u.s" + 0.015*"year" + 0.015*"country" + 0.013*"trade" + 0.012*"great" + 0.011*"d

In [40]:
# display the topics we've found
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    vis_data1 = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(vis_data1)

## Sentiment Analysis

In [41]:
def sentiment_analyzer(sentences, pos_cutoff = 0.7, neg_cutoff = -0.7):
    """
    Given a list of tweets, return those that contain strong positive/negative sentiments
    Args:
        sentences: a list containing the indices of tweets in text_data
        pos_cutoff: tweets with sentiment scores >= pos_cutoff are classified as positive
        neg_cutoff: tweets with sentiment scores <= neg_cutoff are classified as negative
    Returns:
        pos: a list containing the indices of tweets that are classified as positive
        neg: a list containing the indices of tweets that are classified as negative
    """
    analyser  = SentimentIntensityAnalyzer()
    pos, neg  = [], []
    for i in sentences:
        score = analyser.polarity_scores(text_data[i])
        #print("{:-<40} {}".format(s, str(score)))
        if score['compound']   >= pos_cutoff: # positive comments
            pos.append(i)
        elif score['compound'] <= neg_cutoff: # negative comments
            neg.append(i)
    return (pos, neg)

In [42]:
pos, neg = sentiment_analyzer(np.arange(len(text_data)))

In [43]:
len(neg)

1076

In [44]:
len(pos)

2573

In [45]:
len(text_data)

8411

In [46]:
# Positive sentence example
text_data[pos[20]]

'We are all united by the same love of Country the same devotion to family and the same profound faith that America is blessed by the eternal grace of ALMIGHTY GOD! Bound by these convictions we will campaign for every vote &amp; we will WIN the Great State of NEW MEXICO in 2020! '

In [47]:
# Negtive sentence example

text_data[neg[10]]

'The New York Times is now blaming an editor for the horrible mistake they made in trying to destroy or influence Justice Brett Kavanaugh. It wasn’t the editor the Times knew everything. They are sick and desperate losing in so many ways!'

# Movie review example

In [48]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/ilai0/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [49]:
data = dict(pos = movie_reviews.fileids('pos'),
            neg = movie_reviews.fileids('neg'))

In [50]:
def get_review_text (clf,file_id,start=0,end=None):
    words = list(movie_reviews.words(data[clf][file_id]))
    return ' '.join(words[start:end])

In [51]:
# Positive movie example
print(get_review_text('pos',0,end=95))

films adapted from comic books have had plenty of success , whether they ' re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there ' s never really been a comic book like from hell before . for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid ' 80s with a 12 - part series called the watchmen .


In [52]:
# Negative movice example
print(get_review_text('neg',0,start=61,end=191))

critique : a mind - fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn ' t snag this one correctly . they seem to have taken this pretty neat concept , but executed it terribly . so what are the problems with the movie ? well , its main problem is that it ' s simply too jumbled .


In [53]:
def unigram_features (words):
    """
    This is the simplest possible feature representation of a document.

    Each word is a feature.
    """
    return dict((word, True) for word in words)


def extract_features (corpus, file_ids, cls, feature_extractor=unigram_features):
    """
    Turn a set of files all belonging to one class into a list
    of (feature dictionary, cls) pairs, to be used in testing or training
    a classifier.
    """
    return [(feature_extractor(corpus.words(i)), cls) for i in file_ids]


def get_words_from_corpus (corpus, file_ids):

    for file_id in file_ids:
        words = corpus.words(file_id)
        for word in words:
            yield word


In [54]:
# Use 70% as training data and 30% as test data
test_start_index = 700


# Training set
neg_training = extract_features(movie_reviews, data['neg'][:test_start_index], 'neg',
                                feature_extractor=unigram_features)

pos_training = extract_features(movie_reviews, data['pos'][:test_start_index],'pos',
                                feature_extractor=unigram_features)

train_set = pos_training + neg_training

# Test set
neg_test = extract_features(movie_reviews, data['neg'][test_start_index:], 'neg',
                                feature_extractor=unigram_features)

pos_test = extract_features(movie_reviews, data['pos'][test_start_index:],'pos',
                                feature_extractor=unigram_features)

test_set = pos_test + neg_test

## Naive Bayes Classifier

In [56]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_set)

In [57]:
# Conflict comments in one review (difficuly in the task)

# Positive review
print("****Positive review****")
print (get_review_text('pos',0,end=95))

print ('\n. . . . . . \n')

print("****Negative review****")

# Negative review at the same time
print (get_review_text('pos',0,start=-190))

****Positive review****
films adapted from comic books have had plenty of success , whether they ' re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there ' s never really been a comic book like from hell before . for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid ' 80s with a 12 - part series called the watchmen .

. . . . . . 

****Negative review****
, but cinematographer peter deming ( don ' t say a word ) ably captures the dreariness of victorian - era london and helped make the flashy killing scenes remind me of the crazy flashbacks in twin peaks , even though the violence in the film pales in comparison to that in the black - and - white comic . oscar winner martin childs ' ( shakespeare in love ) production design turns the original prague surroundings into one creepy place . even the acting in from hell is solid , with the dre

In [59]:
classifier.classify(train_set[0][0])

'pos'

### Most informative features

In [60]:
classifier.show_most_informative_features()

Most Informative Features
             outstanding = True              pos : neg    =     17.8 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               insulting = True              neg : pos    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
                  avoids = True              pos : neg    =     11.0 : 1.0
               ludicrous = True              neg : pos    =     11.0 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0
                flawless = True              pos : neg    =      9.8 : 1.0
               affecting = True              pos : neg    =      9.7 : 1.0
                riveting = True              pos : neg    =      9.7 : 1.0


In [61]:
from sklearn.metrics import precision_score, recall_score,accuracy_score

def do_evaluation (pairs, pos_label='pos'):
    predicted, actual = zip(*pairs)
    (precision, recall,accuracy) = (precision_score(actual,predicted,pos_label=pos_label),
                                    recall_score(actual,predicted,pos_label=pos_label),
                                    accuracy_score(actual,predicted))

    print_results(precision, recall, accuracy, pos_label)
    

def print_results (precision, recall, accuracy, pos_label):
    banner =  'Evaluation with pos label = %s' % pos_label
    print('\n')
    print(banner)
    print ('=' * len(banner))
    print ('{0:10s} {1:.1f}'.format('Precision',precision*100))
    print ('{0:10s} {1:.1f}'.format('Recall',recall*100))
    print ('{0:10s} {1:.1f}'.format('Accuracy',accuracy*100))


In [62]:
pairs = [(classifier.classify(example), actual)
            for (example, actual) in test_set]
temp =np.array([pairs[i][0]==pairs[i][1]for i in range(len(pairs))])
acc = np.sum(temp)/temp.shape[0]
print(acc)

0.72


## Logistic regression

In [63]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression

classifier = SklearnClassifier(LogisticRegression()).train(train_set)

In [64]:
pairs = [(classifier.classify(example), actual)
            for (example, actual) in test_set]
temp =np.array([pairs[i][0]==pairs[i][1]for i in range(len(pairs))])
acc = np.sum(temp)/temp.shape[0]
print(acc)

0.8816666666666667


# Hotel review example

http://kavita-ganesan.com/entity-ranking-data/#.YoQAR5qZPjg

# Word2vec

In [96]:
# Use truncated SVD to reduce dimension
def reduce_to_k_dim(M, k=2):  
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))

    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components = k, n_iter = n_iters,random_state = 1)
    #a   = svd.fit(M)
    M_reduced = svd.fit_transform(M)
    print("Done.")
    return M_reduced

In [97]:
def get_matrix_of_vectors(wv_from_bin):
    import random
    words = list(wv_from_bin.key_to_index.keys())
    print("Shuffling words ...")
    random.shuffle(words)
    print("Putting %i words into word2Ind and matrix M..." % len(words))
    word2Ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    M = np.stack(M)
    print("Done.")
    return M, word2Ind

In [98]:
def plot_embeddings(M_reduced, word2Ind, words):
    import matplotlib.pyplot as plt
    plt.figure(figsize = (12,8))
    for i,word in enumerate(words):
        x = M_reduced[(word2Ind.get(word),0)]
        y = M_reduced[(word2Ind.get(word),1)]
        plt.scatter(x, y, marker='x', color='red')
        plt.text(x+0.01, y+0.01, word, fontsize=18)

    plt.show()

In [99]:
import gzip
import logging

data_file=_DATA_DIR+"reviews_data.txt.gz"

with gzip.open (_DATA_DIR+'reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

In [100]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))

In [101]:
model = gensim.models.Word2Vec (documents, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

(303494685, 415193550)

In [102]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

[('filthy', 0.8889460563659668),
 ('stained', 0.8082396984100342),
 ('unclean', 0.8022788166999817),
 ('grubby', 0.7974606156349182),
 ('dusty', 0.7963510751724243),
 ('smelly', 0.7929028272628784),
 ('dingy', 0.7699029445648193),
 ('grimy', 0.7560853958129883),
 ('soiled', 0.7536424994468689),
 ('disgusting', 0.7458583116531372)]

In [103]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)

[('courteous', 0.9336776733398438),
 ('friendly', 0.8685598373413086),
 ('cordial', 0.8514259457588196),
 ('curteous', 0.827193558216095),
 ('professional', 0.8222639560699463),
 ('attentive', 0.819309651851654)]

In [104]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)

[('germany', 0.7368361353874207),
 ('canada', 0.7156102657318115),
 ('spain', 0.6930522918701172),
 ('england', 0.6686482429504395),
 ('russia', 0.6552983522415161),
 ('mexico', 0.6542390584945679)]

In [105]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('duvet', 0.7731807827949524),
 ('mattress', 0.7619080543518066),
 ('quilt', 0.7378724217414856),
 ('matress', 0.733214795589447),
 ('blanket', 0.7317253947257996),
 ('pillowcase', 0.730722188949585),
 ('pillowcases', 0.6667075753211975),
 ('foam', 0.6657423377037048),
 ('pillows', 0.664311945438385),
 ('sheets', 0.6577041149139404)]

In [106]:
# get everything related to stuff on the bed
w1 = ['bed']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('beds', 0.4494054913520813),
 ('conditiona', 0.38032662868499756),
 ('paned', 0.3704121708869934),
 ('bedlinens', 0.36778679490089417),
 ('bedgood', 0.36244529485702515),
 ('decker', 0.3618466258049011),
 ('deckers', 0.34756702184677124),
 ('glazing', 0.34623438119888306),
 ('fivestar', 0.34439337253570557),
 ('tempuratures', 0.3399982452392578)]

In [107]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [108]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])

'shower'

In [109]:
model.wv

In [110]:
M, word2Ind = get_matrix_of_vectors(model.wv)

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'key_to_index'

In [83]:
# truncated 2D word2vec matrix
M_reduced = reduce_to_k_dim(M, k=2)

NameError: name 'M' is not defined

In [82]:
# 2D visualization
words = ["bad","free","wonderful","clean","quiet","impressive","terrible","good"]
plot_embeddings(M_reduced, word2Ind, words)

NameError: name 'M_reduced' is not defined

In [79]:
model.wv.distance("happy","glad")

0.5281647145748138

In [80]:
model.wv.distance("happy","bad")

0.7658413201570511

In [81]:
## Analogies: "man : king :: woman : x", what is x?
pprint.pprint(model.wv.most_similar(positive=['man', 'queen'], negative=['woman']))

[('king', 0.794317364692688),
 ('double', 0.7837346196174622),
 ('twin', 0.7092950344085693),
 ('kingsize', 0.6979916095733643),
 ('dbl', 0.6330785751342773),
 ('kingsized', 0.6218348741531372),
 ('queensize', 0.617875337600708),
 ('triple', 0.613437294960022),
 ('murphy', 0.5789287090301514),
 ('bunk', 0.5645344257354736)]
